In [1]:
!pip install ../packages/hermes-0.1.tar.gz

Processing /home/lennon/workspace/audio-seg-data-synth/packages/hermes-0.1.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for hermes: filename=hermes-0.1-py3-none-any.whl size=3453 sha256=19d1f99ee9415a600885d5a43ebd2fa2337e32e57c29714914b8faab7164aded
  Stored in directory: /home/lennon/.cache/pip/wheels/62/26/ad/3afcad542b2d63dc002af3a8213046c04219d1a4d2081f77eb
Successfully built hermes


In [1]:
from hermes.triton.client import TritonClient
from hermes.openvino.client import OpenVinoClient
from musicspeech_class import MusicSpeechClass
from musicspeech_controller import MusicSpeechController
from musicspeech_params import MusicSpeech_Params as music_speech_params

2024-04-15 08:15:01.019196: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-15 08:15:01.630718: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Local

In [12]:
input_path = "/home/lennon/workspace/audio-seg-data-synth/audio/Song.wav"
output_path = "/home/lennon/workspace/audio-seg-data-synth/audio/Song-local.txt"

In [13]:
client = MusicSpeechClass(params=music_speech_params)

In [14]:
controller = MusicSpeechController(client=client, params=music_speech_params)

In [15]:
segments = controller.predict(input_path)

with open(output_path, 'w') as fp:
    fp.write('\n'.join('{}\t{}\t{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in segments))

audio_clip_length_samples is 6154050
(32, 802, 80)
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


## Triton

In [7]:
input_path = "/home/lennon/workspace/audio-seg-data-synth/audio/Song.wav"
output_path = "/home/lennon/workspace/audio-seg-data-synth/audio/Song-triton.txt"

In [8]:
client = TritonClient("music-detection", {"ip": "0.0.0.0", "grpc_port": 8101})

In [9]:
client.metadata

{'name': 'music-detection',
 'versions': ['1'],
 'platform': 'tensorflow_savedmodel',
 'inputs': [{'name': 'mel_input',
   'datatype': 'FP32',
   'shape': ['-1', '802', '80']}],
 'outputs': [{'name': 'time_distributed',
   'datatype': 'FP32',
   'shape': ['-1', '802', '2']}]}

In [10]:
controller = MusicSpeechController(client=client, params=music_speech_params)

In [11]:
segments = controller.predict(input_path)

with open(output_path, 'w') as fp:
    fp.write('\n'.join('{}\t{}\t{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in segments))

audio_clip_length_samples is 6154050
(32, 802, 80)


## OpenVino

In [6]:
input_path = "/home/lennon/workspace/audio-seg-data-synth/audio/Song.wav"
output_path = "/home/lennon/workspace/audio-seg-data-synth/audio/Song-openvino.txt"

In [8]:
client = OpenVinoClient("music-detection", {"ip": "0.0.0.0", "grpc_port": 8101})

In [9]:
client.metadata

{'model_version': 1,
 'inputs': {'mel_input': {'shape': [-1, 802, 80], 'dtype': 'DT_FLOAT'}},
 'outputs': {'functional_3_1/bidirectional_2_1/forward_gru_2_1/strided_slice_1:0': {'shape': [-1,
    1280],
   'dtype': 'DT_FLOAT'},
  'functional_3_1/bidirectional_3_1/forward_gru_3_1/strided_slice_3:0': {'shape': [-1,
    80],
   'dtype': 'DT_FLOAT'},
  'functional_3_1/bidirectional_3_1/backward_gru_3_1/strided_slice_1:0': {'shape': [-1,
    160],
   'dtype': 'DT_FLOAT'},
  'Identity:0': {'shape': [-1, 802, 2], 'dtype': 'DT_FLOAT'},
  'functional_3_1/bidirectional_2_1/backward_gru_2_1/gru_cell_1/add_5:0': {'shape': [-1,
    80],
   'dtype': 'DT_FLOAT'},
  'functional_3_1/bidirectional_2_1/forward_gru_2_1/strided_slice_3:0': {'shape': [-1,
    80],
   'dtype': 'DT_FLOAT'},
  'functional_3_1/bidirectional_3_1/forward_gru_3_1/strided_slice_1:0': {'shape': [-1,
    160],
   'dtype': 'DT_FLOAT'},
  'functional_3_1/bidirectional_3_1/backward_gru_3_1/gru_cell_1/add_5:0': {'shape': [-1,
    80],
  

In [10]:
controller = MusicSpeechController(client=client, params=music_speech_params)

In [11]:
segments = controller.predict(input_path)

with open(output_path, 'w') as fp:
    fp.write('\n'.join('{}\t{}\t{}'.format(round(x[0], 5), round(x[1], 5), x[2]) for x in segments))

audio_clip_length_samples is 6154050


(32, 802, 80)
